<a href="https://colab.research.google.com/github/aproca/MNIST_To_CIFAR10AdvRep/blob/master/encoded_MNIST_trained_on_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
src = list(files.upload().values())[0]

Saving mnist_language_test (2).csv to mnist_language_test (2).csv


In [0]:
from google.colab import files
src = list(files.upload().values())[0]

Saving mnist_language_test (2).csv to mnist_language_test (2).csv


In [0]:
playing = pd.read_csv("mnist_ascii_70000.csv")

In [0]:
print(playing[0:10])

In [0]:
from sklearn.model_selection import train_test_split
import torch
from torch.autograd import Variable
import numpy as np
import os
import io
import requests
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, sampler
from torch import LongTensor, FloatTensor
from sklearn import preprocessing
#from sklearn.datasets import load_iris

#from .normalize import LayerNorm

class ChunkSampler(sampler.Sampler):
   """Samples elements sequentially from some offset.
   Arguments:
       num_samples: # of desired datapoints
       start: offset where we should start selecting from
   """
   def __init__(self, num_samples, start = 0):
       self.num_samples = num_samples
       self.start = start
   def __iter__(self):
       return iter(range(self.start, self.start + self.num_samples))
   def __len__(self):
       return self.num_samples
NUM_TRAIN = 100
NUM_TEST = 100

#datatrain = pd.read_csv("mnist_ascii_70000_2 (2).csv")
datatrain = pd.read_csv("mnist_language_test (2).csv")

#datatrain = datatrain[0:10]

y = datatrain['784'].values              # number labels
                               # removes number labels and image number label
X = datatrain.drop(columns=['784']).values


BATCH_SIZE = 16

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

#train_norm = X_train[X_train[0:784]]
#test_norm = X_test[X_test[0:784]]
#train_norm = X_train.tolist
#test_norm = X_test.tolist

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train = std_scale.transform(X_train)
X_test  = std_scale.transform(X_test)

#X_train = preprocessing.scale(X_train)
#X_train = preprocessing.normalize(X_train)
#X_test = preprocessing.scale(X_test)
#X_test = preprocessing.normalize(X_test)

# Normalize Training Data 
#std_scale = preprocessing.StandardScaler().fit(X_train)
#X_train_norm = std_scale.transform(X_train)
#Converting numpy array to dataframe
#X_train = pd.DataFrame(X_train_norm) 
#X_train.update(training_norm_col)
#print (X_train)
# Normalize Testing Data by using mean and SD of training set
#X_test_norm = std_scale.transform(X_test)
#testing_norm_col = pd.DataFrame(X_test_norm, index=test_norm.index, columns=test_norm.columns) 
#X_test.update(testing_norm_col)
#print (X_train.head())


#print(len(X_train))
X_train = X_train.reshape(8499,1,28,28)


# data type is long
torch_X_train = torch.from_numpy(X_train).type(torch.FloatTensor) # image arrays
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # label array

# # create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

dataset = TensorDataset( FloatTensor(torch_X_train), LongTensor(torch_y_train) )

loader = DataLoader(dataset, batch_size= BATCH_SIZE ,sampler=ChunkSampler(NUM_TRAIN,0), shuffle = False)

dataset2 = TensorDataset(FloatTensor(torch_X_test),LongTensor(torch_y_test))
loader2 = DataLoader(dataset2, batch_size = BATCH_SIZE, shuffle = False)

# Pytorch train and test sets
#train = torch.utils.data.TensorDataset(torch_X_train, torch_y_train)
#test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                          sampler=ChunkSampler(NUM_TRAIN,0), num_workers=8)
#train_loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size = BATCH_SIZE, shuffle = False)

In [0]:
# Try word2vec and see what it gives

corp = X

def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corp)

AttributeError: ignored

In [0]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 16
num_epochs = 1000
learning_rate = 0.001

#train_dataset = train
#test_dataset = test


In [0]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.sequence_length = sequence_length
        self.input_size = input_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
      
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))# out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out
      

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [0]:
def fit(model, loader):
    #optimizer = torch.optim.(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    error = nn.CrossEntropyLoss()
    EPOCHS = 1000
    #model.train()
    total_step = len(loader)
    correct = 0
    total = 0
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(loader):
            #print(type(X_batch))
            #var_X_batch = Variable(X_batch).float()    # it's a problem here i think
            #X_batch = X_batch.float()
            X_batch = X_batch.reshape(-1, sequence_length, input_size).to(device)
            #X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            #print(X_batch.size())

            
            output = model(X_batch)
            loss = error(output, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(output.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
            
            if (batch_idx) % 50 == 0:
              print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, EPOCHS, batch_idx+1, total_step, loss.item()))
              print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / BATCH_SIZE))


            # Total correct predictions
            #predicted = torch.max(output.data, 1)[1] 
            #correct += (predicted == var_y_batch).sum()
            #print(correct)
            #if batch_idx % 50 == 0:
                #print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                #    epoch, batch_idx*len(X_batch), len(loader.dataset), 100.*batch_idx / len(loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                
              

In [0]:
fit(model= model, loader=loader)

Epoch [1/1000], Step [1/7], Loss: 2.2946
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [2/1000], Step [1/7], Loss: 2.2930
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [3/1000], Step [1/7], Loss: 2.2914
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [4/1000], Step [1/7], Loss: 2.2898
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [5/1000], Step [1/7], Loss: 2.2883
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [6/1000], Step [1/7], Loss: 2.2868
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [7/1000], Step [1/7], Loss: 2.2853
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [8/1000], Step [1/7], Loss: 2.2838
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [9/1000], Step [1/7], Loss: 2.2824
Test Accuracy of the model on the 10000 test images: 6.25 %
Epoch [10/1000], Step [1/7], Loss: 2.2810
Test Accuracy of the model on the 10000 test imag

In [0]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


Test Accuracy of the model on the 10000 test images: 90.0 %


In [0]:
def fit2():
  total_step = len(train_loader)
  for epoch in range(num_epochs):
      for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
          var_X_batch = Variable(X_batch).float() # i think error is here
          var_y_batch = Variable(y_batch)
        
         # Forward pass
          outputs = model(var_X_batch)
          loss = criterion(output, var_y_batch)
        
          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
        
        # Total correct predictions
          predicted = torch.max(output.data, 1)[1] 
          correct += (predicted == var_y_batch).sum()
         #print(correct)
          if batch_idx % 50 == 0:
              print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                  epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))


In [0]:
import glob

# Importing drive method from colab for accessing google drive
from google.colab import drive

# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Viewing the data in the folder present from the drive
!ls "/content/drive/My Drive/MNIST_Text_Class"

 checkpoint.pth			 MNIST_CNN.ipynb
'Copy of encoding.ipynb'	'mnist_language_test (2).csv'
'Copy of mnist.ipynb'		 names
'Copy of train2_working.ipynb'	 Pretrained_Names18.ipynb
 LSTM_Arch.ipynb		 train2.ipynb
 mnist_ascii_70000.csv


In [0]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [0]:
model = load_checkpoint(glob.glob('checkpoint.pth'))
print(model)

AttributeError: ignored